# Тема “Обучение с учителем”

---
### Задание 3*

Вызовите документацию для класса `RandomForestRegressor`, найдите информацию об атрибуте **feature_importances_**.
С помощью этого атрибута найдите сумму всех показателей важности, установите, какие два признака показывают наибольшую важность.

#### Загружаем необходимые библиотеки и устанавливаем настройки среды для работы

In [1]:
# Импортируем необходимые библиотеки
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pylab import rcParams

In [2]:
# Отключаем предупреждения
import warnings

warnings.filterwarnings('ignore')

In [3]:
# Магическая команда Jupyter Notebook нужна для того, чтобы графики отображались прямо в ноутбуке, а не в отдельном окне:
%matplotlib inline

In [4]:
# Магическая команда позволяет рисовать графики в формате `svg`, т.е. scalable vector graphics - масштабируемая векторная
# графика. Это придаёт изображениям большую чёткость.
%config InlineBackend.figure_format = 'svg'

#### Загружаем и смотрим датасет

In [5]:
# Функция load_boston загружает датасет.
from sklearn.datasets import load_boston

# Загружаем его в переменную boston.
boston = load_boston()

In [6]:
# Этот датасет представлен в виде словаря.
# Cмотрим ключи этого словаря:
boston.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [7]:
# Данные о недвижимости хранятся в массиве по ключу "data".
data = boston["data"]

data.shape

(506, 13)

In [8]:
# Каждая строка соответствует какому-то объекту (объекту недвижимости), а столбцы - каким-то его характеристикам.
# Названия признаков хранятся в массиве по ключу "feature_names":
feature_names = boston["feature_names"]

feature_names

array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [9]:
# Массив с целевыми значениями (ценами на недвижимость) можно получить по ключу "target":
target = boston["target"]

target[:10]

array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9])

#### Создаём несколько таблиц DataFrame

In [10]:
# Создаём несколько таблиц DataFrame для более удобного хранения данных.
# В таблице X будут храниться признаки.
# В качестве названий для столбцов возьмём массив feature_names:
X = pd.DataFrame(data, columns=feature_names)

X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [11]:
# Посмотрим информацию по таблице "Х"
X.info()
# Пропущенных значений в таблице нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


In [12]:
# Создадим таблицу "y", в которую запишем целевые значения:
y = pd.DataFrame(target, columns=["price"])

y.info()
# Пропущенных значений в таблице нет

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   506 non-null    float64
dtypes: float64(1)
memory usage: 4.1 KB


#### Разбиваем получившиеся датафреймы на тренировочные *(X_train, y_train)* и тестовые *(X_test, y_test)*

In [13]:
# Разбиение данных на тренировочную и тестовую выборку выполняем с помощью функции train_test_split
# из модуля sklearn.model_selection.

from sklearn.model_selection import train_test_split

In [14]:
# С помощью параметра test_size указываем, какую часть данных выделяем под тест.
# Если указать число из отрезка  [0, 1], то оно будет интерпретироваться как доля тестовых объектов.
# Если указать число большее или равное 1, то это будет число объектов в тестовой выборке.
# Указание параметра random_state делает результат разбиения повторяемым (одинаковым) при каждом выполнении функции.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=True, random_state=42)

#### Построение модели RandomForestRegressor

In [15]:
# Загружаем модель линейной регрессии из библиотеки sklearn

from sklearn.ensemble import RandomForestRegressor

Несколько параметров данной модели:

* `n_estimators` - число деревьев в ансамбле
* `max_features` - максимальное число признаков, которое может быть использовано при построении каждого дерева
* `max_depth` - максимальная глубина дерева
* `random_state` - Управляет как случайностью бутстреппинга выборок, используемых при построении деревьев (если ``bootstrap=True``) и выборкой признаков, которые будут учитываться при поиске наилучшего разделения в каждом узле      признаков для рассмотрения при поиске наилучшего разбиения в каждом узле. При задании числа приводит к повторяемости результатов (по умолчанию = None)


In [16]:
# Создаём модель

model = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)

#### Обучение модели

In [17]:
# Обучаем модель на тренировочных данных (используя все признаки).
# чтобы обучить модель, используем метод .fit, в который передаём тренировочную выборку:
model.fit(X_train, y_train.values[:, 0])

RandomForestRegressor(max_depth=12, n_estimators=1000, random_state=42)

#### Выводим документацию для RandomForestRegressor

In [18]:
# Выводим документацию для RandomForestRegressor
RandomForestRegressor?

#### Attributes

**feature_importances_** : ndarray of shape (n_features,)
    The impurity-based feature importances.
    The higher, the more important the feature.
    The importance of a feature is computed as the (normalized)
    total reduction of the criterion brought by that feature.  It is also
    known as the Gini importance.

    *Warning*: impurity-based feature importances can be misleading for
    high cardinality features (many unique values). See
    :func:`sklearn.inspection.permutation_importance` as an alternative.


#### Показатели важности модели `RandomForestRegressor`

In [19]:
# Выведем для просмотра показатели важности модели
print(f'Показатели важности модели:\n{model.feature_importances_}')

Показатели важности модели:
[0.0338027  0.00320578 0.00619517 0.00128683 0.01201835 0.35808131
 0.01471098 0.05965106 0.00469447 0.01251055 0.01922967 0.01187371
 0.46273942]


#### Сумма всех показателей важности

In [20]:
# Суммируем все показатели важности модели RandomForestRegressor
print(f'Сумма показателей важности = {model.feature_importances_.sum()}')

Сумма показателей важности = 1.0000000000000002


In [21]:
# Для удобства создаём датафрейм с именами показателей и их значениями
feature_importances = pd.DataFrame(model.feature_importances_, feature_names)

# Отсортируем показатели от большего к меньшему
feature_importances.sort_values(by=0, ascending=False)

,0
LSTAT,0.462739
RM,0.358081
DIS,0.059651
CRIM,0.033803
PTRATIO,0.019230
AGE,0.014711
TAX,0.012511
NOX,0.012018
B,0.011874
INDUS,0.006195


#### Наибольшую важность показывают признаки `LSTAT` и `RM`

In [22]:
# Выведем два признака, показывающих наибольшую важность.
feature_importances.nlargest(2, 0)

,0
LSTAT,0.462739
RM,0.358081
